In [ ]:
import pandas as pd
import numpy as np
from json import loads
import pandas as pd
import psycopg2 
import csv
#from urllib import parse, request
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import sys
sys.path.append("/Users/bhatnagarakshit10/Documents/DSE/akbhatna/dse203-group-project/query_capability")

In [128]:
#Extract the features from DB
conn = psycopg2.connect("dbname='SQLBook' user='postgres' host='localhost' password='postgres' password='postgres'")
cur = conn.cursor()

#categories: dataframe that contains categories information from Asterixdb (nodeId and 5 levels of categories)
#categories.to_csv('categories.csv')

#reviewFeatures: dataframe that contains review features from Postgres
#reviewFeatures.to_csv('reviewFeatures.csv')

In [129]:
#Function to get review length
def rev_len(x):
    #print (x)
    x['reviewlen']=len(str(x['txt']))
    return x

#Function to get review length
def rev_len(x):
    #print (x)
    x['reviewlen']=len(str(x['txt']))
    return x

#Get number of sentences
def count_sentences(x):
    cnt=0
    num_exc=0
    for i in str(x):
        if i in ['.','?','!']:
            cnt=cnt+1
    for i in str(x):
        if i in ['?','!']:
            num_exc=num_exc+1
    return cnt,num_exc 

#Get average word length
def avg_word_len(x):
    word_list = str(x).split(' ')
    nwords = len(word_list)
    tot_len=0
    for w in word_list:
        tot_len=tot_len+len(w)
    return tot_len/nwords

In [130]:
cur.execute("SELECT r.reviewID,r.asin, cast(r.overall as int) as rating, cast(trim(leading '[' from substring(helpful from 0 for position(',' in helpful))) as int) as votesForReview,cast(trim(trailing ']' from substring(helpful from position(',' in helpful)+2)) as int) as outOf,reviewCount.cnt as reviewerCount,bookCount.cnt as bookreviewCount,EXTRACT(day from current_date - r.reviewTime) age,pr.nodeID,reviewtext,summary FROM reviews r LEFT OUTER JOIN (SELECT reviewID, count(*) cnt FROM reviews GROUP BY reviewID) reviewCount ON r.reviewID = reviewCount.reviewID LEFT OUTER JOIN (SELECT asin, count(*) cnt FROM reviews GROUP BY asin) bookCount ON r.asin = bookCount.asin LEFT OUTER JOIN Products pr ON r.asin = pr.asin")
rows = cur.fetchall()

df = pd.DataFrame(rows,columns=['reviewID','asin','rating','helpfulness','outOf','reviewerCount','bokReviewCount','reviewAge','nodeID','reviewtext','summary'])

#Combining summary and review text
df['txt']=df['reviewtext']+df['summary']
df.drop(['reviewtext','summary'],axis=1,inplace=True)

#Get review length
df['reviewlen']=0
df=df.apply(lambda x: rev_len(x),axis=1)

#Get # of words, word length, # of sentences, ARI index
df['numwords']=0
df['avgwordlen']=0
df['num_sen']=0
df['num_exc']=0
df['ARI']=0

def numwords(x):
    x['numwords']=len(str(x['txt']).split(' '))
    x['avgwordlen'] = avg_word_len(x['txt'])
    x['num_sen'],x['num_exc'] = count_sentences(x['txt'])
    x['ARI'] = 4.71*(len(str(x['txt']))/float(x['numwords']+1)) + 0.5*(x['numwords']/float(x['num_sen']+1)) - 21.43
    return x
df=df.apply(lambda x: numwords(x),axis=1)

#Extract just the feature set from original df
new_df = df[['rating','helpfulness','outOf','reviewerCount','bokReviewCount','reviewAge','reviewlen','numwords','avgwordlen','num_sen','num_exc','ARI']]

In [131]:
#Training data is the reviews with more than 1 outof
train_df=new_df[new_df['outOf']!=0]

#Prediction data is the reviews which dont have any outof
test_df=new_df[new_df['outOf']==0]

#Get the training labels
y_train = train_df['helpfulness']/train_df['outOf']

#Drop the label data from train
train_df.drop(['helpfulness','outOf'],axis=1,inplace=True)

#Split the data into train, validate, test
X_train, X_test, y_train, y_test = train_test_split(train_df, y_train, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=23)

/Users/bhatnagarakshit10/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [132]:
#Normalize the features
scaler = StandardScaler()
scaler.fit(X_train)
StandardScaler(copy=True, with_mean=True, with_std=True)
X_train_n = scaler.transform(X_train)
X_test_n = scaler.transform(X_test)
X_val_n = scaler.transform(X_val)

In [133]:
# Fit linear regression
reg = LinearRegression()
reg.fit(X_train_n,y_train.values)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [134]:
# Predict the output for train and validate
y_train_pred = reg.predict(X_train_n)
y_val_pred=reg.predict(X_val_n)

In [135]:
#Check the error for train and validate
mean_squared_error(y_train.values,y_train_pred)
mean_squared_error(y_val.values,y_val_pred)

0.13921201051198817

In [136]:
#Predict for test data
y_test_pred=reg.predict(X_test_n)

#Check the error
mean_squared_error(y_test.values,y_test_pred)

0.1383579551103622

In [137]:
#Actual prediction on unknown labels
test_df.drop(['helpfulness','outOf'],axis=1,inplace=True)

test_df_n = scaler.transform(test_df) 

y_test_df=reg.predict(test_df_n)

#Store the redicted help ratio
test_df['help_ratio']=y_test_df

/Users/bhatnagarakshit10/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/bhatnagarakshit10/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [138]:
#Create help ratio in original df
new_df['help_ratio']=new_df['helpfulness']/new_df['outOf']

# Join 
new_df=new_df.join(test_df['help_ratio'],how='left',rsuffix='1')

#Get the final ratio
def final_ratio(x):
    if np.isnan(x['help_ratio']):
        x['help_ratio']=x['help_ratio1']
    return x

new_df=new_df.apply(lambda x: final_ratio(x),axis=1)
new_df.drop('help_ratio1',inplace=True,axis=1)

/Users/bhatnagarakshit10/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [139]:
df=df.join(new_df['help_ratio'],how='left')
df['rating*help_ratio']=df['rating']*df['help_ratio']
df_grouped = df.groupby('asin').sum()

def get_count(x):
    x['count']=len(df[df['asin']==x['asin']])
    return x

df_grouped['count']=0
df_grouped=df_grouped.reset_index().apply(lambda x: get_count(x),axis=1)

df_grouped['overall_rating1'] = (df_grouped['rating*help_ratio']/df_grouped['count']) + np.log(df_grouped['count'])
#df_grouped=df_grouped.sort_values(by='overall_rating1',ascending=False)

In [140]:
#Get categories mapping, index to category name
cats_map=[]
with open('numcats.txt','r') as f:
    reader=csv.reader(f,delimiter='\t')
    for num,cat in reader:
        cats_map.append(cat)
        
#Get books mapping, index to asin
asin = np.load('asin.npy')

In [141]:
#Get total purchase count for each asin
cur.execute("""select p.asin,count(1) from customers c,
orders o, orderlines ol,products p
where c.customerid = o.customerid
and o.orderid =ol.orderid
and ol.productid =p.productid
group by p.asin""")
rows = cur.fetchall()

purchase_df = pd.DataFrame(rows,columns=['asin','purchase_cnt'])

In [142]:
df_grouped=pd.merge(df_grouped,purchase_df,on='asin',how='left')

In [143]:
df_grouped['overall_rating1'] = df_grouped['overall_rating1']+np.log(df_grouped['purchase_cnt'])

# Asterixdb

In [15]:
class QueryResponse:
    def __init__(self, raw_response):
        self._json = loads(raw_response)

        self.requestID = self._json['requestID'] if 'requestID' in self._json else None
        self.clientContextID = self._json['clientContextID'] if 'clientContextID' in self._json else None
        self.signature = self._json['signature'] if 'signature' in self._json else None
        self.results = self._json['results'] if 'results' in self. _json else None
        self.metrics = self._json['metrics'] if 'metrics' in self._json else None

class AsterixConnection:
    def __init__(self, server = 'http://localhost', port = 19002):
        self._server = server
        self._port = port
        self._url_base = self._server +':'+ str(port)

    def query(self, statement, pretty=False, client_context_id=None):
        endpoint = '/query/service'

        url = self._url_base + endpoint

        payload = {
            'statement': statement,
            'pretty': pretty
        }

        if client_context_id:
            payload['client_context_id'] = client_context_id

        data = parse.urlencode(payload).encode("utf-8")
        req = request.Request(url, data)
        response = request.urlopen(req).read()

        return QueryResponse(response)

In [19]:
#Get category information, grouped by nodeID
asterix_conn = AsterixConnection()
response = asterix_conn.query("""USE TinySocial;
    SELECT 
        nodeID, 
        (SELECT gp.cat.category.nested.level_1, gp.cat.category.nested.nested.level_2, gp.cat.category.nested.nested.nested.level_3, gp.cat.category.nested.nested.nested.nested.level_4, gp.cat.category.nested.nested.nested.nested.nested.level_5 FROM gp) AS categories
    FROM ClassificationInfo c
    GROUP BY c.nodeID AS nodeID
    GROUP AS gp(c as cat)""")
# """

#Parse result from asterixdb and create a dictionary {nodeID: [[lvl1,lvl2,lvl3,lvl4,lvl5],[lvl1,lvl2,..],..]}
max_cat = 0
categories = []

for i in range(len(response.results)):
    if len(response.results[i]['categories']) > max_cat:
        max_cat = len(response.results[i]['categories'])
    c = []
    for x in range(len(response.results[i]['categories'])):
        c.append([cats_map.index(response.results[i]['categories'][x][l]) for l in response.results[i]['categories'][x]])
    categories.append([response.results[i]['nodeID'],c])
    
categories = {i[0]:i[1] + [[0,0,0,0,0]]*(max_cat-len(i[1])) for i in categories}

# Categories Matrix

In [144]:
#Get nodeid to asin mapping
cur.execute("""select nodeid,asin from products""")
rows = cur.fetchall()
nodeid_asin = pd.DataFrame(rows,columns=['nodeid','asin'])
nodeid_asin = nodeid_asin.set_index('asin')

In [111]:
#Reorder categories with the nodeid to asin mapping (from postgres) and index to asin mapping (from asin.npy)
categories_indexed = []

for i in range(max_cat):
    c = []
    for a in asin:
        c.append(categories[int(nodeid_asin.ix[a.decode()]['nodeid'])][i])
    categories_indexed.append(c)

NameError: name 'max_cat' is not defined

# Get seasonal information

In [145]:
cur.execute("""select asin,
          round(100*sum(case when month >= 3 and month < 6 then numunits else 0 end)/sum(numunits),2) as spring,
          round(100*sum(case when month >= 6 and month < 9 then numunits else 0 end)/sum(numunits),2) as summer,
          round(100*sum(case when month >= 9 and month < 12 then numunits else 0 end)/sum(numunits),2) as fall,
          round(100*sum(case when (month = 12 or month < 3) then numunits else 0 end)/sum(numunits),2) as winter,
          cast(max(fullprice) as decimal) fullprice,
          case when max(isinstock) = 'Y' then 1 else 0 end isinstock
          from 
          (select asin, EXTRACT(MONTH FROM orderdate) as month, case when l.numunits = 0 then 0.00001 else l.numunits end as numunits, fullprice, isinstock
          from customers c, orders o, orderlines l, products p
          where c.customerid = o.customerid
          and o.orderid = l.orderid
          and l.productid = p.productid
          ) as temp
          group by asin""")
# """
rows = cur.fetchall()

In [146]:
season = pd.DataFrame(rows,columns=['asin','spring','summer','fall','winter','fullprice','isinstock'])

In [147]:
list_asin_names=list(np.load('asin.npy'))

In [148]:
list_asin_names=[i.decode() for i in list_asin_names]

In [149]:
def get_idx_from_asin(asin_list):
	list_idx = []
	for asin in asin_list:
		try:
			list_idx.append(list_asin_names.index(asin))
		except ValueError:
			pass
	return list_idx

In [150]:
asin_final = []

for i in asin:
    if i.decode() in df_grouped.asin.values:
        row = [i for i in df_grouped[df_grouped.asin==i.decode()][['asin','overall_rating1','count']].values[0]]
    else:
        row = [0,0]
        
    if i.decode() in season.asin.values:
        row += [i for i in season[season.asin==i.decode()][['spring','summer','fall','winter','fullprice','isinstock']].values[0]]
    else:
        row += [0,0,0,0,0,'']
        
    asin_final.append(row)

In [151]:
def get_asin_index(x):
    l=[]
    l.append(x['asin'])
    x['asin_index']=get_idx_from_asin(l)[0]
    return x failed to coerce slice entry of type list to integer

In [152]:
df_asin=pd.DataFrame(asin_final,columns=['asin','overall_rating1','count','spring','summer','fall','winter','fullprice','isinstock'])

In [153]:
df_asin['asin_index']=-1
df_asin=df_asin.apply(lambda x: get_asin_index(x),axis=1)

In [154]:
df_asin.sort_values(by='overall_rating1',ascending=False)

,asin,overall_rating1,count,spring,summer,fall,winter,fullprice,isinstock,asin_index
2025,0007179286,15.514313,69,21.95,19.17,32.06,26.82,179.00,1,2025
2864,0007256817,15.050645,179,13.31,23.28,56.13,7.28,29.00,0,2864
1874,0007107927,14.303064,371,1.31,0.79,46.19,51.71,14.00,0,1874
2845,0002007770,14.300077,6001,11.11,0.00,55.56,33.33,340.00,1,2845
3220,0007236360,14.241021,532,3.13,89.06,3.39,4.43,10.00,1,3220
2782,0007386648,14.163302,9062,0.00,100.00,0.00,0.00,34.00,1,2782
747,000612609X,14.097544,184,9.52,37.00,40.29,13.19,10.00,0,747
1019,0007145454,14.031931,2,0.86,62.50,27.87,8.77,32.00,1,1019
2655,0006135129,13.828675,59,8.58,13.67,10.19,67.56,39.00,1,2655
1980,0000029831,13.828219,5,21.03,19.41,21.72,37.85,16.00,1,1980


In [156]:
df_asin[df_asin.index==2845]

,asin,overall_rating1,count,spring,summer,fall,winter,fullprice,isinstock,asin_index
2845,0002007770,14.300077,6001,11.11,0.00,55.56,33.33,340.00,1,2845


In [ ]:
2782, 2845, 1031,  647, 3278, 3971, 3080, 3001, 1912, 3504

In [ ]:
1524,930,3703,1019,2290,2056,275,2347,2984,361

In [172]:
rating_indexed = np.array(asin_final)[:,1]

In [175]:
rating_indexed[rating_indexed==0] = np.min([float(i) for i in rating_indexed[rating_indexed != 0]])/100

In [177]:
len(rating_indexed)

3990

In [178]:
np.save("rating_indexed.npy",rating_indexed)

In [38]:
np.save('season_price_instock_indexed.npy',np.array(asin_final, dtype=np.float)[:,2:])

ValueError: could not convert string to float: 